<a href="https://colab.research.google.com/github/wy-go/google-research/blob/master/spreadsheet_coder/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data
Load data from [[UI]](https://console.cloud.google.com/storage/browser/spreadsheet_coder)


## Copy files from GCS with gsutil

In [12]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'sheetcoder'
!gcloud config set project {project_id}

Updated property [core/project].


In [13]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil -m cp -r \
  "gs://spreadsheet_coder/enron/" \
  .
  
# Print the result to make sure the transfer worked.
# !cat /sheetcoder/data/formulas_0_5.tf_record

Copying gs://spreadsheet_coder/enron/formulas_5_10.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_10_15.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_0_5.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_15_20.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_20_25.tf_record...
Copying gs://spreadsheet_coder/enron/formulas_25_30.tf_record...
| [6/6 files][  3.9 GiB/  3.9 GiB] 100% Done  14.9 MiB/s ETA 00:00:00           
Operation completed over 6 objects/3.9 GiB.                                      


## Read the TFRecord files

In [15]:
import tensorflow as tf
import os

data_dir = 'enron/'
tfrecord_files = [data_dir + filename for filename in os.listdir(data_dir)]
raw_dataset = tf.data.TFRecordDataset(tfrecord_files)

# Show the first 3 records.
for raw_record in raw_dataset.take(3):
  print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b"\n\xdci\n\x12\n\tcol_index\x12\x05\x1a\x03\n\x01\x13\n\x0e\n\x06doc_id\x12\x04\n\x02\n\x00\n\x1b\n\x06header\x12\x11\n\x0f\n\rContract Cost\n\xdad\n\x0ccontext_data\x12\xc9d\n\xc6d\n\xc3d$R[-1]C[-10]$userEnteredValue=empty$$R[-1]C[-9]$userEnteredValue=empty$$R[-1]C[-8]$userEnteredValue=empty$$R[-1]C[-7]$userEnteredValue=empty$$R[-1]C[-6]$userEnteredValue=empty$$R[-1]C[-5]$userEnteredValue=empty$$R[-1]C[-4]$userEnteredValue=empty$$R[-1]C[-3]$userEnteredValue=empty$$R[-1]C[-2]$userEnteredValue=empty$$R[-1]C[-1]$userEnteredValue=empty$$R[-1]C[0]$userEnteredValue=empty$$R[-1]C[1]$userEnteredValue=empty$$R[-1]C[2]$userEnteredValue=empty$$R[-1]C[3]$userEnteredValue=empty$$R[-1]C[4]$userEnteredValue=empty$$R[0]C[-10]$userEnteredValue=str:New$$R[0]C[-9]$userEnteredValue=doub:37165.0$$R[0]C[-8]$userEnteredValue=str:Whitewing$$R[0]C[-7]$userEnteredValue=str:$4.5MM DASHed$$R[0]C[-6]$userEnteredValue=str:N$$R[0]C[-5]$userEnteredValue=str:N/A$$R[0]C[-4]$u

In [17]:
# Create a description of the features.
feature_description = {
    'table_id': tf.io.FixedLenFeature([], tf.int64),
    'doc_id': tf.io.FixedLenFeature([], tf.string),
    'record_index': tf.io.FixedLenFeature([], tf.int64),
    'col_index': tf.io.FixedLenFeature([], tf.int64),
    'formula': tf.io.FixedLenFeature([], tf.string),
    'formula_token_list': tf.io.FixedLenFeature([], tf.string),
    'ranges': tf.io.FixedLenFeature([], tf.string),
    'computed_value': tf.io.FixedLenFeature([], tf.string),
    'header': tf.io.FixedLenFeature([], tf.string),
    'context_header': tf.io.FixedLenFeature([], tf.string),
    'context_data': tf.io.FixedLenFeature([], tf.string)
}

def _parse_example(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

parsed_dataset = raw_dataset.map(_parse_example)

parsed_dataset

<MapDataset shapes: {col_index: (), computed_value: (), context_data: (), context_header: (), doc_id: (), formula: (), formula_token_list: (), header: (), ranges: (), record_index: (), table_id: ()}, types: {col_index: tf.int64, computed_value: tf.string, context_data: tf.string, context_header: tf.string, doc_id: tf.string, formula: tf.string, formula_token_list: tf.string, header: tf.string, ranges: tf.string, record_index: tf.int64, table_id: tf.int64}>

In [18]:
# Show the first 3 observations in datasets.
for parsed_record in parsed_dataset.take(3):
  print(repr(parsed_record))

{'col_index': <tf.Tensor: shape=(), dtype=int64, numpy=19>, 'computed_value': <tf.Tensor: shape=(), dtype=string, numpy=b'doub:83.41666666666667'>, 'context_data': <tf.Tensor: shape=(), dtype=string, numpy=b'$R[-1]C[-10]$userEnteredValue=empty$$R[-1]C[-9]$userEnteredValue=empty$$R[-1]C[-8]$userEnteredValue=empty$$R[-1]C[-7]$userEnteredValue=empty$$R[-1]C[-6]$userEnteredValue=empty$$R[-1]C[-5]$userEnteredValue=empty$$R[-1]C[-4]$userEnteredValue=empty$$R[-1]C[-3]$userEnteredValue=empty$$R[-1]C[-2]$userEnteredValue=empty$$R[-1]C[-1]$userEnteredValue=empty$$R[-1]C[0]$userEnteredValue=empty$$R[-1]C[1]$userEnteredValue=empty$$R[-1]C[2]$userEnteredValue=empty$$R[-1]C[3]$userEnteredValue=empty$$R[-1]C[4]$userEnteredValue=empty$$R[0]C[-10]$userEnteredValue=str:New$$R[0]C[-9]$userEnteredValue=doub:37165.0$$R[0]C[-8]$userEnteredValue=str:Whitewing$$R[0]C[-7]$userEnteredValue=str:$4.5MM DASHed$$R[0]C[-6]$userEnteredValue=str:N$$R[0]C[-5]$userEnteredValue=str:N/A$$R[0]C[-4]$userEnteredValue=str:EWS

## Statistics from paper

<div>
  <h6 align=center>Dataset</h6>

  <table border="1" class="dataframe">
    <thead>
      <tr align=center>
        <th></th>
        <th>Train</th>
        <th>Validation</th>
        <th>Test</th>
        <th>Total</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>#Samples</th>
        <td>178K</td>
        <td>41K</td>
        <td>33K</td>
        <td>252K</td>
      </tr>
    </tbody>
  </table>
  <br />
  <table border="1" class="dataframe">
    <thead>
      <tr align=center>
        <th align=left>Sketch Length</th>
        <th>2</th>
        <th>3</th>
        <th>4-5</th>
        <th>6-7</th>
        <th>8+</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>Distribution</th>
        <td>55%</td>
        <td>18%</td>
        <td>13%</td>
        <td>9%</td>
        <td>5%</td>
      </tr>
    </tbody>
  </table>
  <br />
  <table border="1" class="dataframe">
    <thead>
      <tr align=center>
        <th></th>
        <th>Spreadsheet Functions</th>
        <th>Type</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>1</th>
        <td>ADD(+)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>2</th>
        <td>MINUS(-)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>3</th>
        <td>MULPTIPLY(*)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>4</th>
        <td>DIV(/)</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>5</th>
        <td>UPLUS</td>
        <td>Operator</td>
      </tr>
      <tr align=center>
        <th align=left>6</th>
        <td>UMINUS</td>
        <td>Operator</td>
      </tr>      
      <tr align=center>
        <th align=left>7</th>
        <td>SUM</td>
        <td>Math</td>
      </tr>
      <tr align=center>
        <th align=left>8</th>
        <td>ABS</td>
        <td>Math</td>
      </tr>
      <tr align=center>
        <th align=left>9</th>
        <td>LN</td>
        <td>Math</td>
      </tr>
      <tr align=center>
        <th align=left>10</th>
        <td>AVERAGE</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>11</th>
        <td>MIN</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>12</th>
        <td>MAX</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>13</th>
        <td>COUNT</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>14</th>
        <td>COUNTA</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>15</th>
        <td>STDEV</td>
        <td>Statistical</td>
      </tr>
      <tr align=center>
        <th align=left>16</th>
        <td>DAY</td>
        <td>Date</td>
      </tr>
      <tr align=center>
        <th align=left>17</th>
        <td>WEEKDAY</td>
        <td>Date</td>
      </tr>
    </tbody>
  </table>
</div>

<br />

<div>
  <h6 align=center>Result</h6>
  <table border="1" class="dataframe">
    <thead>
      <tr align=center>
        <th align=left>Dataset</th>
        <th>Top-1</th>
        <th>Top-5</th>
        <th>Top-10</th>
      </tr>
    </thead>
    <tbody>
      <tr align=center>
        <th align=left>Enron</th>
        <td>29.8%</td>
        <td>41.8%</td>
        <td>48.5%</td>
      </tr>
      <tr align=center>
        <th align=left>Google Sheets</th>
        <td>42.51%</td>
        <td>54.41%</td>
        <td>58.57%</td>
      </tr>
    </tbody>
  </table>
</div>

# Model![model architecture](https://raw.github.com/wy-go/google-research/master/spreadsheet_coder/spreadsheetcoder_model_architecture.png)


In [19]:
# Clone the entire repo.
!git clone https://github.com/google-research/google-research.git

Cloning into 'google-research'...
remote: Enumerating objects: 36727, done.
remote: Counting objects: 100% (1415/1415), done.
remote: Compressing objects: 100% (872/872), done.
remote: Total 36727 (delta 615), reused 1152 (delta 518), pack-reused 35312
Receiving objects: 100% (36727/36727), 296.83 MiB | 29.16 MiB/s, done.
Resolving deltas: 100% (19755/19755), done.
Checking out files: 100% (12418/12418), done.


In [20]:
!ls google-research/spreadsheet_coder/

bert_modeling.py  mobilebert_modeling.py  model_utils.py
constants.py	  model.py		  README.md


In [ ]:
# Exclude intended target
